In [1]:
import numpy as np
import pandas as pd
import torch
from pytorch_pretrained_bert.tokenization import BertTokenizer, WordpieceTokenizer
from pytorch_pretrained_bert.modeling import BertForPreTraining, BertPreTrainedModel, BertModel, BertConfig, BertForMaskedLM, BertForSequenceClassification
import string
import nltk
from nltk.corpus import stopwords
import os

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
df = pd.read_csv("discharge_notes_with_medication_full_text_18APR.csv")
df = df[df["admission_notes"].notna()]
discharge_notes = df["discharge_notes"].fillna("").tolist()
admission_notes = df["admission_notes"].fillna("").tolist()

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
table = str.maketrans({key: None for key in string.punctuation})
stops = set(stopwords.words("english"))

In [154]:
def seg_list(original, stride=256, length=512):
    result_list = []
    n = 0
    total = len(original)
    center_list = []
    index_list = np.arange(total)
    while n+length<total:
        result_list.append(original[n:n+length])
        center_list.append((n+n+length)/2)
        n+=stride
        
    if n>0 and n<total:
        result_list.append(original[n:])
        center_list.append((n+total)/2)
    close_index = [np.argmin(np.abs(center_list-x)) for x in index_list]
    position_index = index_list-[r*stride for r in close_index]
    return result_list, close_index, position_index

In [176]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
with torch.no_grad():
    doc_features = []
    for i in range(len(admission_notes)):
        text = admission_notes[i].translate(table)
        text = ' '.join([word for word in text.split() if word not in stops])
        tokenized_text = tokenizer.tokenize(text)
        text_length = len(tokenized_text)
        print("Processing document No."+str(i)+" with length "+str(text_length))
        if(text_length==0):
            doc_features.append(np.zeros((1,768)))
        elif text_length<=512:
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
            segments_ids = [1]*len(indexed_tokens)
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])
            encoded_layers, output_layers = model(tokens_tensor, segments_tensors)
            doc_features.append(encoded_layers[-1][0].numpy())
            print(doc_features[i].shape)
        else:
            seg_text, token_mapper, position_mapper = seg_list(tokenized_text)
            feature_list = []
            for sub_text in seg_text:
                indexed_tokens = tokenizer.convert_tokens_to_ids(sub_text)
                segments_ids = [1]*len(indexed_tokens)
                tokens_tensor = torch.tensor([indexed_tokens])
                segments_tensors = torch.tensor([segments_ids])
                encoded_layers, output_layers = model(tokens_tensor, segments_tensors)
                feature_list.append(encoded_layers[-1][0].numpy())
            doc_features.append(np.array([feature_list[token_mapper[j]][position_mapper[j]] for j in range(text_length)]))
            print(doc_features[i].shape)
        
    doc_features = np.array(doc_features)
    np.save("doc_features_bert.npy", doc_features)
    print("complete!")    
                

Processing document No.0 with length 385
(385, 768)
Processing document No.1 with length 864
(864, 768)
Processing document No.2 with length 627
(627, 768)
Processing document No.3 with length 814
(814, 768)
Processing document No.4 with length 925
(925, 768)
Processing document No.5 with length 433
(433, 768)
Processing document No.6 with length 612
(612, 768)
Processing document No.7 with length 479
(479, 768)
Processing document No.8 with length 743
(743, 768)
Processing document No.9 with length 728
(728, 768)
Processing document No.10 with length 822
(822, 768)
Processing document No.11 with length 1024
(1024, 768)
Processing document No.12 with length 479
(479, 768)
Processing document No.13 with length 375
(375, 768)
Processing document No.14 with length 872
(872, 768)
Processing document No.15 with length 702
(702, 768)
Processing document No.16 with length 652
(652, 768)
Processing document No.17 with length 0
Processing document No.18 with length 570
(570, 768)
Processing doc

Processing document No.308 with length 363
(363, 768)
Processing document No.309 with length 2034
(2034, 768)
Processing document No.310 with length 1053
(1053, 768)
Processing document No.311 with length 551
(551, 768)
Processing document No.312 with length 702
(702, 768)
Processing document No.313 with length 654
(654, 768)
Processing document No.314 with length 454
(454, 768)
Processing document No.315 with length 835
(835, 768)
Processing document No.316 with length 799
(799, 768)
Processing document No.317 with length 933
(933, 768)
Processing document No.318 with length 360
(360, 768)
Processing document No.319 with length 334
(334, 768)
Processing document No.320 with length 147
(147, 768)
Processing document No.321 with length 742
(742, 768)
Processing document No.322 with length 899
(899, 768)
Processing document No.323 with length 768
(768, 768)
Processing document No.324 with length 838
(838, 768)
Processing document No.325 with length 1411
(1411, 768)
Processing document No

Processing document No.612 with length 787
(787, 768)
Processing document No.613 with length 804
(804, 768)
Processing document No.614 with length 762
(762, 768)
Processing document No.615 with length 0
Processing document No.616 with length 561
(561, 768)
Processing document No.617 with length 594
(594, 768)
Processing document No.618 with length 1058
(1058, 768)
Processing document No.619 with length 638
(638, 768)
Processing document No.620 with length 0
Processing document No.621 with length 731
(731, 768)
Processing document No.622 with length 522
(522, 768)
Processing document No.623 with length 334
(334, 768)
Processing document No.624 with length 653
(653, 768)
Processing document No.625 with length 816
(816, 768)
Processing document No.626 with length 876
(876, 768)
Processing document No.627 with length 1024
(1024, 768)
Processing document No.628 with length 267
(267, 768)
Processing document No.629 with length 0
Processing document No.630 with length 738
(738, 768)
Processin

(1411, 768)
Processing document No.920 with length 869
(869, 768)
Processing document No.921 with length 779
(779, 768)
Processing document No.922 with length 452
(452, 768)
Processing document No.923 with length 1008
(1008, 768)
Processing document No.924 with length 952
(952, 768)
Processing document No.925 with length 896
(896, 768)
Processing document No.926 with length 343
(343, 768)
Processing document No.927 with length 1124
(1124, 768)
Processing document No.928 with length 0
Processing document No.929 with length 1141
(1141, 768)
Processing document No.930 with length 654
(654, 768)
Processing document No.931 with length 0
Processing document No.932 with length 0
Processing document No.933 with length 1000
(1000, 768)
Processing document No.934 with length 1078
(1078, 768)
Processing document No.935 with length 661
(661, 768)
Processing document No.936 with length 379
(379, 768)
Processing document No.937 with length 814
(814, 768)
Processing document No.938 with length 1446
(

Processing document No.1221 with length 1392
(1392, 768)
Processing document No.1222 with length 1595
(1595, 768)
Processing document No.1223 with length 92
(92, 768)
Processing document No.1224 with length 475
(475, 768)
Processing document No.1225 with length 479
(479, 768)
Processing document No.1226 with length 1069
(1069, 768)
Processing document No.1227 with length 650
(650, 768)
Processing document No.1228 with length 1215
(1215, 768)
Processing document No.1229 with length 303
(303, 768)
Processing document No.1230 with length 340
(340, 768)
Processing document No.1231 with length 251
(251, 768)
Processing document No.1232 with length 224
(224, 768)
Processing document No.1233 with length 1203
(1203, 768)
Processing document No.1234 with length 3119
(3119, 768)
Processing document No.1235 with length 680
(680, 768)
Processing document No.1236 with length 572
(572, 768)
Processing document No.1237 with length 1133
(1133, 768)
Processing document No.1238 with length 997
(997, 768

...

(439, 768)
Processing document No.18699 with length 1318
(1318, 768)
Processing document No.18700 with length 896
(896, 768)
Processing document No.18701 with length 1905
(1905, 768)
Processing document No.18702 with length 725
(725, 768)
Processing document No.18703 with length 529
(529, 768)
Processing document No.18704 with length 0
Processing document No.18705 with length 836
(836, 768)
Processing document No.18706 with length 794
(794, 768)
Processing document No.18707 with length 850
(850, 768)
Processing document No.18708 with length 390
(390, 768)
Processing document No.18709 with length 1628
(1628, 768)
Processing document No.18710 with length 497
(497, 768)
Processing document No.18711 with length 940
(940, 768)
Processing document No.18712 with length 1032
(1032, 768)
Processing document No.18713 with length 266
(266, 768)
Processing document No.18714 with length 39
(39, 768)
Processing document No.18715 with length 174
(174, 768)
Processing document No.18716 with length 660

Processing document No.18991 with length 421
(421, 768)
Processing document No.18992 with length 1841
(1841, 768)
Processing document No.18993 with length 669
(669, 768)
Processing document No.18994 with length 508
(508, 768)
Processing document No.18995 with length 322
(322, 768)
Processing document No.18996 with length 448
(448, 768)
Processing document No.18997 with length 586
(586, 768)
Processing document No.18998 with length 635
(635, 768)
Processing document No.18999 with length 579
(579, 768)
Processing document No.19000 with length 0
Processing document No.19001 with length 64
(64, 768)
Processing document No.19002 with length 680
(680, 768)
Processing document No.19003 with length 2600
(2600, 768)
Processing document No.19004 with length 603
(603, 768)
Processing document No.19005 with length 741
(741, 768)
Processing document No.19006 with length 0
Processing document No.19007 with length 126
(126, 768)
Processing document No.19008 with length 0
Processing document No.19009 

Processing document No.19300 with length 0
Processing document No.19301 with length 28
(28, 768)
Processing document No.19302 with length 0
Processing document No.19303 with length 122
(122, 768)
Processing document No.19304 with length 0
Processing document No.19305 with length 7
(7, 768)
Processing document No.19306 with length 7
(7, 768)
Processing document No.19307 with length 0
Processing document No.19308 with length 4
(4, 768)
Processing document No.19309 with length 0
Processing document No.19310 with length 0
Processing document No.19311 with length 8
(8, 768)
Processing document No.19312 with length 0
Processing document No.19313 with length 0
Processing document No.19314 with length 0
Processing document No.19315 with length 0
Processing document No.19316 with length 0
Processing document No.19317 with length 11
(11, 768)
Processing document No.19318 with length 9
(9, 768)
Processing document No.19319 with length 0
Processing document No.19320 with length 0
Processing docume

Processing document No.19627 with length 0
Processing document No.19628 with length 0
Processing document No.19629 with length 62
(62, 768)
Processing document No.19630 with length 0
Processing document No.19631 with length 55
(55, 768)
Processing document No.19632 with length 0
Processing document No.19633 with length 7
(7, 768)
Processing document No.19634 with length 7
(7, 768)
Processing document No.19635 with length 0
Processing document No.19636 with length 3
(3, 768)
Processing document No.19637 with length 7
(7, 768)
Processing document No.19638 with length 0
Processing document No.19639 with length 0
Processing document No.19640 with length 0
Processing document No.19641 with length 217
(217, 768)
Processing document No.19642 with length 0
Processing document No.19643 with length 150
(150, 768)
Processing document No.19644 with length 399
(399, 768)
Processing document No.19645 with length 0
Processing document No.19646 with length 0
Processing document No.19647 with length 24

MemoryError: 

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()
with torch.no_grad():
    doc_features = []
    for i in range(len(admission_notes)):
        text = admission_notes[i].translate(table)
        text = ' '.join([word for word in text.split() if word not in stops])
        tokenized_text = tokenizer.tokenize(text)
        text_length = len(tokenized_text)
        print("Processing document No."+str(i)+" with length "+str(text_length))
        if(text_length==0):
            doc_features.append(np.zeros((1,768)))
        elif text_length<=512:
            indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
            segments_ids = [1]*len(indexed_tokens)
            tokens_tensor = torch.tensor([indexed_tokens])
            segments_tensors = torch.tensor([segments_ids])
            encoded_layers, output_layers = model(tokens_tensor, segments_tensors)
            doc_features.append(encoded_layers[-1][0].numpy())
            print(doc_features[i].shape)
        else:
            seg_text, token_mapper, position_mapper = seg_list(tokenized_text)
            feature_list = []
            for sub_text in seg_text:
                indexed_tokens = tokenizer.convert_tokens_to_ids(sub_text)
                segments_ids = [1]*len(indexed_tokens)
                tokens_tensor = torch.tensor([indexed_tokens])
                segments_tensors = torch.tensor([segments_ids])
                encoded_layers, output_layers = model(tokens_tensor, segments_tensors)
                feature_list.append(encoded_layers[-1][0].numpy())
            doc_features.append(np.array([feature_list[token_mapper[j]][position_mapper[j]] for j in range(text_length)]))
            print(doc_features[i].shape)
        
    doc_features = np.array(doc_features)
    np.save("doc_features_bert.npy", doc_features)
    print("complete!")    

In [177]:
len(doc_features)

19710

In [191]:
index = 0
n=0
while n+500<len(doc_features):
    t=doc_features[n:n+500]
    np.save("doc_features_bert"+str(index)+".npy", t)
    index+=1
    n+=500
t=doc_features[n:]
np.save("doc_features_bert"+str(index)+".npy", t)

In [190]:
np.save("doc_features_bert1.npy", t)


In [194]:
doc_features[0].dtype

dtype('float32')

In [198]:
import h5py

In [199]:
h5f = h5py.File('doc_features_bert.h5', 'w')

In [200]:
h5f.create_dataset('features_bert', data=doc_features)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [203]:
doc_features.shape

(19710,)

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('discharge_notes_with_medication_full_text_18APR.csv')
labels = np.array(df.iloc[:,14:-1])
del df


In [13]:
def label_smoothing(l,smoothing = 0.2,num_classes = 8):
    l = l.cpu().numpy()
    x=l.astype(np.float32)
    label_sum = np.sum(x,axis = 1)
    one_replace = 1-smoothing
    zero_replace = label_sum*smoothing/(num_classes-label_sum)

    x[x==1] = one_replace

    for i in range(x.shape[0]):
        temp = x[i]
        temp[temp==0] = zero_replace[i]

        x[i]=temp
    x = torch.tensor(x).cuda()
    return x

In [8]:
label_smoothing(labels)

array([[0.02857143, 0.02857143, 0.02857143, ..., 0.02857143, 0.8       ,
        0.02857143],
       [0.8       , 0.02857143, 0.02857143, ..., 0.02857143, 0.02857143,
        0.02857143],
       [0.8       , 0.8       , 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667],
       ...,
       [0.06666667, 0.06666667, 0.8       , ..., 0.06666667, 0.06666667,
        0.8       ],
       [0.02857143, 0.02857143, 0.02857143, ..., 0.02857143, 0.8       ,
        0.02857143],
       [0.8       , 0.8       , 0.06666667, ..., 0.06666667, 0.06666667,
        0.06666667]], dtype=float32)

In [9]:
import torch

In [10]:
x = torch.tensor(labels)

In [14]:
label_smoothing(x)

AssertionError: Torch not compiled with CUDA enabled